In [1]:
import os
import sys
import numpy as np
import nibabel as nib
import pydicom
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from numpy import zeros, newaxis
from scipy.stats import norm
import cv2
import scipy.signal as signal
from pydicom.pixel_data_handlers.util import apply_color_lut
from skimage.filters import threshold_multiotsu
from skimage.filters import threshold_otsu, threshold_local
import scipy.io
import pandas as pd
import SimpleITK as sitk

# Util functions

In [182]:
def plot_data(data, axis=0):
    if axis==0:
        for i in range(0, len(data[0,0,:])):
            img=data[:,:,i]
            plt.imshow(img)
            plt.show()
    elif axis==1:
        for i in range(0, len(data[0,:,0])):
            img=data[:,i,:]
            plt.imshow(img)
            plt.show()
    elif axis==2:
        for i in range(0, len(data[:,0,0])):
            img=data[i,:,:]
            plt.imshow(img)
            plt.show()
            
def ch_coordinates(bottom, top, nchannels, vox_res=39.06):
#     ch_coord=np.linspace(pstart, pstop, nchannels).astype('int')
    unitvec=top-bottom
    unitvec=unitvec/np.linalg.norm(unitvec)
    
    ch_coord=np.zeros((nchannels,3))
    for i in range(nchannels):
        ch_coord[i,:]=np.round(bottom+(i*unitvec*50/vox_res))
    
    return (ch_coord).astype('int'), unitvec



## Reading atlas labels

In [183]:
labels_path='./atlas_labels.rtf'
labels=[]
anats=[]
with open(labels_path) as f:
    lines = f.readlines()
    for i,line in enumerate(lines):
        if i>6 and i%2==1:
            if i==7:
                anat= ' '.join(line.split('\\cf')[1].split()[8:])
#                 label=[line.split('\\cf')[1].split()[0], anat]
                anats.append(anat.split("\"")[1])
                labels.append(int(line.split('\\cf')[1].split()[0]))
#                 print(label)
            else:
                anat=' '.join(line.split()[7:])
#                 label=[line.split()[0], anat]
                anats.append(anat.split("\"")[1])
                labels.append(int(line.split()[0]))
#                 print(label)
                
df=pd.DataFrame()
df["Labels"]=labels
df["Anatomical Regions"]=anats
df

,Labels,Anatomical Regions
0,0,Clear Label
1,1,corticofugal tract and corona radiata
2,3,Subthalamic nucleus
3,4,Molecular cell layer of the cerebellum
4,5,"Cerebellum, unspecified"
...,...,...
218,444,Temporal association cortex
219,448,"Secondary visual area, medial part"
220,500,Endopiriform nucleus
221,501,"Amygdaloid area, unspecified"


In [34]:
# atlas_path='/Users/mri_registration/SAMRI/WHS_SD_rat_atlas_v4_pack/WHS_SD_rat_atlas_v4.nii.gz'
# nii_atlas=nib.load(atlas_path)
# atlas=np.asanyarray(nii_atlas.dataobj)

In [205]:
# Probe map for right-hemi implantation
siteMap = np.flip(np.reshape(scipy.io.loadmat('right_hemi_sitemap.mat')['siteMap'], -1))

# rEO_05 Electrode Map

## Elastic mapping

In [68]:
tpath="/Users/eminhanozil/electrode-localization/mri/rEO_05/output_Composite.h5"
niipath="/Users/eminhanozil/electrode-localization/mri/rEO_05/sub-rEO_05_ses-presurgery_task-rest_acq-TurboRARE_T2w-ind_10_resampled.nii.gz"
niipath2="/Users/eminhanozil/electrode-localization/mri/rEO_05/WHS_SD_rat_T2star_v1.01.nii.gz"

In [69]:
read_result=sitk.ReadTransform(tpath)
img=sitk.ReadImage(niipath)  # Moving image
img2=sitk.ReadImage(niipath2)  # Fixed image

In [154]:
# Generating indeces in moving_img for each index in fixed_img
# Just load it from files next time no need to run it, it takes time to compute
nx, ny, nz= (512, 1024, 512)

moving_coordinates=np.empty((nx*ny*nz, 3))
fixed_coordinates=np.empty((nx*ny*nz, 3))

i=0
for x in range(nx):
    for y in range(ny):
        for z in range(nz):
            idx=[x,y,z]
            fixedpnt=img2.TransformIndexToPhysicalPoint(idx)
            movingpnt=read_result.TransformPoint(fixedpnt)
            movingidx=img.TransformPhysicalPointToIndex(movingpnt)
            fixed_coordinates[i,:]=idx
            moving_coordinates[i,:]=movingidx
            i=i+1

In [181]:
fixedcorr_path="/Users/eminhanozil/Dropbox (Yanik Lab)/Localization Manuscript 2024/RAT DATA/rEO_05/MRI_ANALYSED/elastic/fixed_img-idx.npy"
np.save(fixedcorr_path, fixed_coordinates)

movingcorr_path="/Users/eminhanozil/Dropbox (Yanik Lab)/Localization Manuscript 2024/RAT DATA/rEO_05/MRI_ANALYSED/elastic/moving_presurgeryresampled_img-idx.npy"
np.save(movingcorr_path, moving_coordinates)

In [175]:
#Bottom index at presurgery_resampled
idx=[165,185,304]
bottom_index =((moving_coordinates[:,0] == idx[0]) & (moving_coordinates[:,1] == idx[1]) & (moving_coordinates[:,2] == idx[2]))

#Top index at presurgery_resampled
idx=[170,271,312]
top_index =((moving_coordinates[:,0] == idx[0]) & (moving_coordinates[:,1] == idx[1]) & (moving_coordinates[:,2] == idx[2]))

In [180]:
print(fixed_coordinates[top_index])
print(fixed_coordinates[bottom_index])

print(np.linalg.norm(fixed_coordinates[top_index]-fixed_coordinates[bottom_index])*0.039)
print(np.linalg.norm(np.array([165,185,304])-np.array([170,271,312]))*0.039)

[[301. 578. 403.]]
[[306. 567. 317.]]
3.38694286931445
3.3741198852441507


In [197]:
atlas_path='/Users/eminhanozil/Dropbox (Yanik Lab)/Localization Manuscript 2024/RAT DATA/rEO_05/MRI_ANALYSED/elastic/WHS_SD_rat_atlas_v4.nii.gz'
nii_atlas=nib.load(atlas_path)
atlas=np.asanyarray(nii_atlas.dataobj)
print(atlas.shape)

(512, 1024, 512)


In [207]:
electrode_trajectory=np.zeros(np.shape(atlas))
bottom=fixed_coordinates[bottom_index]-1
top=fixed_coordinates[top_index]-1
ch_coord,_=ch_coordinates(bottom, top, 64)

root='/Users/eminhanozil/Dropbox (Yanik Lab)/Localization Manuscript 2024/RAT DATA/rEO_05/MRI_ANALYSED/elastic/'
animal_id="rEO_05"
with open(root+animal_id+'-Elastic_chMap.txt', 'w') as f:
    for idx, coord in enumerate(ch_coord):
        print(coord)
        label=atlas[coord[0],coord[1],coord[2]]
        electrode_trajectory[coord[0]-1:coord[0]+1,coord[1]-1:coord[1]+1,coord[2]-1:coord[2]+1]=1
        anat_region=df["Anatomical Regions"][df["Labels"]==label].values[0]
        line="CH:"+str(siteMap[idx]-1)+" in "+anat_region+' Segment: '+str(label) + " atlas coord: " + str(coord)
        print(line)
        f.write(line)
        f.write('\n')

[305 566 316]
CH:62 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [305 566 316]
[305 566 317]
CH:60 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [305 566 317]
[305 566 319]
CH:63 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [305 566 319]
[305 566 320]
CH:61 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [305 566 320]
[305 567 321]
CH:0 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [305 567 321]
[305 567 322]
CH:58 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [305 567 322]
[305 567 324]
CH:1 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [305 567 324]
[304 567 325]
CH:59 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: [304 567 325]
[304 567 326]
CH:2 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229 atlas coord: 

## Affine mapping

In [6]:
atlas_path='/Users/eminhanozil/Dropbox (Yanik Lab)/Localization Manuscript 2024/RAT DATA/rEO_05/MRI_ANALYSED/rEO_05-affine-atlas.nii.gz'
nii_atlas=nib.load(atlas_path)
atlas=np.asanyarray(nii_atlas.dataobj)
print(atlas.shape)

In [32]:
#rEO-05 right hemi
# top=np.array([307, 572, 434])
# bottom=np.array([304, 565, 314])

top=np.array([170,271,312])
bottom=np.array([165,185,304])

ch_coord, unitvec=ch_coordinates(bottom, top, 64)
electrode_trajectory=np.zeros(np.shape(atlas))

In [34]:
animal_id="rEO_05"
with open(animal_id+'-Affine_chMap.txt', 'w') as f:
    for idx, coord in enumerate(ch_coord):
        print(coord)
        label=atlas[coord[0],coord[1],coord[2]]
        electrode_trajectory[coord[0]-1:coord[0]+1,coord[1]-1:coord[1]+1,coord[2]-1:coord[2]+1]=1
        anat_region=df["Anatomical Regions"][df["Labels"]==label].values[0]
        line="CH:"+str(siteMap[idx])+" in "+anat_region+' Segment: '+str(label) + " atlas coord: " + str(coord)
        print(line)
        f.write(line)
        f.write('\n')
    #     print('Segment: '+str(label))

[165 185 304]
CH:63 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0 atlas coord: [165 185 304]
[165 186 304]
CH:61 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0 atlas coord: [165 186 304]
[165 188 304]
CH:64 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0 atlas coord: [165 188 304]
[165 189 304]
CH:62 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0 atlas coord: [165 189 304]
[165 190 304]
CH:1 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0 atlas coord: [165 190 304]
[165 191 305]
CH:59 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0 atlas coord: [165 191 305]
[165 193 305]
CH:2 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0 atlas coord: [165 193 305]
[166 194 305]
CH:60 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0 atlas coord: [166 194 305]
[166 195 305]
CH:3 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 2

# rEO_06 Electrode Map

In [27]:
animal_id="rEO_06"
with open(animal_id+'-Affine_chMap.txt', 'w') as f:
    for idx, coord in enumerate(ch_coord):
        label=atlas[coord[0],coord[1],coord[2]]
        electrode_trajectory[coord[0]-1:coord[0]+1,coord[1]-1:coord[1]+1,coord[2]-1:coord[2]+1]=1
        anat_region=df["Anatomical Regions"][df["Labels"]==label].values[0]
        line="Site:"+str(siteMap[idx])+" in "+anat_region+' Segment: '+str(label)
        print(line)
        f.write(line)
        f.write('\n')
    #     print('Segment: '+str(label))

Site:63 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:61 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:64 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:62 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:1 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:59 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:2 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:60 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:3 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:57 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:4 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:58 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:5 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:55 in Latero

In [34]:
nii_trajectory=nib.Nifti1Image(electrode_trajectory, nii_atlas.affine)
nib.save(nii_trajectory, 'rEO_05_electrode_trajectory_segmentation.nii.gz')

In [58]:
eo06_20240201_bottom=np.array([108, 113, 200])
eo06_20240201_top=np.array([ 118, 182, 193])

ch_coord, unitvec=ch_coordinates(eo06_20240201_bottom, eo06_20240201_top, 64, 60)

In [31]:
atlas_path='/Users/mri_registration/SAMRI/samri_output/rEO_06/segmentation_rEO-06.nii.gz'
nii_eo06_atlas=nib.load(atlas_path)
eo06_atlas=np.asanyarray(nii_eo06_atlas.dataobj)

In [60]:
eo06_path='/Users/mri_registration/SAMRI/samri_output/rEO_06/bids/sub-rEO_06/ses-presurgery/anat/sub-rEO_06_ses-presurgery_task-rest_acq-TurboRAREreg_T2w-ind_00_resampled.nii.gz'
nii_eo06=nib.load(eo06_path)
eo06_img=np.asanyarray(nii_eo06.dataobj)

In [59]:
atlas=eo06_atlas
electrode_trajectory=np.zeros(np.shape(atlas))
for idx, coord in enumerate(ch_coord):
    label=atlas[coord[0],coord[1],coord[2]]
    electrode_trajectory[coord[0]-1:coord[0]+1,coord[1]-1:coord[1]+1,coord[2]-1:coord[2]+1]=1
    anat_region=df["Anatomical Regions"][df["Labels"]==label].values[0]
    print("Site:"+str(siteMap[idx])+" in "+anat_region+' Segment: '+str(label))
#     print('Segment: '+str(label))

Site:63 in Posterior thalamic nucleus Segment: 230.0
Site:61 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:64 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:62 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:1 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:59 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:2 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:60 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:3 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:57 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:4 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:58 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:5 in Laterodorsal thalamic nucleus, ventrolateral part Segment: 229.0
Site:55 in Laterodorsal thalamic nucleus

In [61]:
nii_trajectory=nib.Nifti1Image(electrode_trajectory, nii_eo06.affine)
nib.save(nii_trajectory, '/Users/mri_registration/SAMRI/samri_output/rEO_06/bids/sub-rEO_06/ses-postsurgery20240201/electrode_trajectory.nii.gz')